# Create dataset for collected data

In [1]:
import json
import os
import pickle
import re

import pandas as pd

In [2]:
bbf_path = '../results/bbf'

### Read quotes to list

In [3]:
quotes = []

for file in os.listdir(bbf_path):
    if file.endswith(".txt"):
        with open(os.path.join(bbf_path, file)) as f:
            lines = f.readlines()
            for line in lines:
                quote = json.loads(line)
                quotes.append(quote)

### Create dataset

In [4]:
df = pd.DataFrame(quotes)

In [5]:
df.shape

(303411, 5)

In [6]:
df.head()

,quote,author,source,character,tag
0,"«Я люблю тебя», — как часто эти слова срываютс...",В. А. Редер,Пещера Лейхтвейса,Аделина Барберини,"[Брак , Любовь , Молодость , Самообман]"
1,"За окошком камеры моей Клен стоит, едва скриви...","[В. А. Кондратьев, З. Н. Политов]",Говорят погибшие герои,Денисенко,Война
2,"Нас было 21. Стояли насмерть. Погибаем, но не ...","[В. А. Кондратьев, З. Н. Политов]",Говорят погибшие герои,Нестеренко,Война
3,"Прощайте все, кто был дорог мне, а море, Черно...","[В. А. Кондратьев, З. Н. Политов]",Говорят погибшие герои,моряк,Война
4,Когда я в атаке в дыму и в огне Иду по колено ...,"[В. А. Кондратьев, З. Н. Политов]",Говорят погибшие герои,Усов,"[Война, Любовь]"


### Normalize columns

In [7]:
def normalize_column(tag):
    if type(tag) == list:
        return list(set(
            [' '.join(item.split()) for item in tag]
        ))
    elif type(tag) == str:
        return [tag, ]
    else:
        return []

In [8]:
df['author'] = df['author'].apply(normalize_column)

In [9]:
df['character'] = df['character'].apply(normalize_column)

In [10]:
df['tag'] = df['tag'].apply(normalize_column)

### Convert tags to lowercase

In [11]:
def create_list_column(tag):
    if type(tag) == list:
        return [item.lower() for item in tag]
    elif type(tag) == str:
        return [tag.lower(), ]
    else:
        return list()

In [12]:
df.tag = df.tag.apply(create_list_column)

In [13]:
df = df.rename(columns={"tag": "tags"})

### Save raw data

In [14]:
with open(os.path.join(bbf_path, 'raw_bbf.pickle'), 'wb') as f:
    pickle.dump(df, f)

## Prepare data for Sarcasm/Irony Detection task

### Mark all dialogs

In [15]:
def is_dialog(text):
    if (
        (len(re.findall(r'—', text)) >= 2 and re.match(r'—', text))
        or 
        (len(re.findall(r'–', text)) >= 2 and re.match(r'–', text))
        or 
        (len(re.findall(r'−', text)) >= 2 and re.match(r'−', text))
        or 
        (len(re.findall(r'-', text)) >= 2 and re.match(r'-', text))
    ):
        return 1
    else:
        return 0

In [16]:
def is_character_in_text(text, characters):
    text = text.lower()
    for character in [char.lower() for char in characters]:
        try:
            if re.findall(character, text):
                return True
        except:
            ...
    return False

In [17]:
df['is_dialog'] = df['quote'].apply(is_dialog)

In [18]:
for ind, row in df.iterrows():
    if len(row.character) > 1:
         if is_character_in_text(row.quote, row.character):
                df.at[ind, 'is_dialog'] = 1

### Get specific labeles

In [19]:
def get_tags_by_topic(tag, topic_list):
    return True if set(tag).intersection(topic_list) else False

In [20]:
labels = [
    'ирония', 'ироничные', 'самоирония', 
    'сарказм', 'саркастичные',
    # addition tags 
    'насмешки', 'издевательство', 'сатира', 
    'остроумие', 'черный юмор',
]

In [21]:
df['target'] = df['tags'].apply(
    lambda x: get_tags_by_topic(x, labels)
)

In [22]:
df.target = df.target.fillna(0)
df.target = pd.to_numeric(df.target, errors='coerce').astype(int)

In [23]:
df[df.target == True].shape

(21576, 7)

## Save data

In [24]:
with open(os.path.join(bbf_path, 'bbf.pickle'), 'wb') as f:
    pickle.dump(df, f)